<a href="https://colab.research.google.com/github/fannix/timeseries_generation/blob/master/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Wed Aug 28 17:30:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [2]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 204Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 88.2Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 356Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:04, 102Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 202Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 126Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 125Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [3]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
# file_name = "shakespeare.txt"
file_name = "abcnews-date-text.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:19<00:00, 79.26s/it]


dataset has 16012725 tokens
Training...
[10 | 18.86] loss=3.52 avg=3.52
[20 | 31.54] loss=3.37 avg=3.44
[30 | 44.14] loss=3.39 avg=3.42
[40 | 56.81] loss=3.47 avg=3.44
[50 | 69.50] loss=3.14 avg=3.37
[60 | 82.23] loss=3.12 avg=3.33
[70 | 94.88] loss=3.31 avg=3.33
[80 | 107.63] loss=3.27 avg=3.32
[90 | 120.31] loss=3.21 avg=3.31
[100 | 132.97] loss=3.08 avg=3.28
[110 | 145.72] loss=3.19 avg=3.27
[120 | 158.36] loss=3.07 avg=3.26
[130 | 171.09] loss=3.24 avg=3.25
[140 | 183.73] loss=3.18 avg=3.25
[150 | 196.41] loss=3.03 avg=3.23
[160 | 209.04] loss=2.85 avg=3.21
[170 | 221.67] loss=2.96 avg=3.19
[180 | 234.32] loss=3.17 avg=3.19
[190 | 246.95] loss=3.03 avg=3.18
[200 | 259.66] loss=3.06 avg=3.17
======== SAMPLE 1 ========
 in kenyanese political system
20120110223,crocodiles wont be banned for six years under law minister
20110223,concern over health service training
20110223,climate change and warming will affect
201,coroners warned underfunding to be tackled in new years
20110223,coun

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## bad cases.

gpt2.generate(sess, run_name='run1', length=1000, prefix="20131231")

```
20131231,newcastle city council withdraws part of council funding
20121231,newcastle magistrates court recognises mother's death
20121231,new life for spittler after wingarang in nsw
20121231,new town to be built on pikemeade
20121231,news exchange friday october 31
20121231,nicholls tasmanian cattle industry and
20121231,no town drinking water
```
gpt2.generate(sess, run_name='run1', destination_path="test", length=1000, prefix="2013")

```
20131027,man arrested over fire in leicestershire
20071027,man charged over death of paddleboarder
20071027,man charged over alleged fisherman murder
20071027,man charged with murder of stepfather
20071027,man to face court on attempted murder charge
20071027,man to stand trial over alleged arson
20071027,man to stand trial over attempted murder
20071027,mayor welcomes coastal development plannin
```
gpt2.generate(sess, run_name='run1', destination_path="test", length=1000, prefix="0723")

```
0723,administration prepares for retirement
20080701,administrator to be sworn in
20080701,afghan forces kill 4 militants
20080701,afl to see off attack on london
20080701,afl to look at new ethics rules
20080701,afl to review australian firefighters
20080701,alp says it is not time to axe reds
```
gpt2.generate(sess, run_name='run1', destination_path="test", length=1000, prefix="2020")

```
202020519,soderling fields changes anything but boring
2012020519,soderling hwy at the heart of australias obsession
2012020519,sorenstam and michael mcconnon
2012020519,sorenstam and michael mcconnon
2012020519,south east sa and bangladesh
2012020519,south korea and australia to meet over nuclear issues
```

```
20200202,sunday 19th october
20140512,shipwrecked indigenous man found dead in fitzroy
20140512,suspected wto drug trafficker comes to light
20140512,sydney fireworks show saves town from demolition
20140512,tas country hour 21 october 2015
20140512,work to start on new royal hotel
20140512,zimbabwe interview
20141019,15m pledge for new adelaide hospital
20141019,2015 city of port macquarie's first short film
20141019,2015 vic police station chase
20141019,2015 tasmanian exports welcome to the world
20141019,2015 tasmanian cattle export guide
```
gpt2.generate(sess, run_name='run1', nsamples=32, batch_size=16, destination_path="test", prefix=" 2018")

```
20030903
20040407
20081013
20090309
20091124
20100723
20110217
20110218
20110603
20110709
20120121
20130101
20140202
20140927
20150312
20150313
20150415
20150427
20150811
20160506
20161210
20170414
20170429
20170521
20170527
20170818
20170826
20171224
20171225
20181027
20190611
20190612
```

gpt2.generate(sess, run_name='run1', nsamples=32, batch_size=16, destination_path="test", prefix="halloween")
```
20030208
20030210
20030225
20030226
20040420
20041012
20080827
20090421
20090422
20100811
20100813
20100826
20100930
20110203
20110204
20110227
20110301
20110702
20111014
20120410
20120706
20120707
20130401
20130405
20130507
20130702
20140608
20140617
20140618
20140903
20141130
20150709
20150727
20150810
20150831
20160521
20160528
20161031
20170506
20170507
20180503
20180504
20181224
====================
halloween
halloween 2015
halloween a league
halloween and the people behind it
halloween at the airport
halloween at the st josie village
halloween display
halloween in sa
halloween in the goth
halloween lights up swedish town
halloween party still a shock for families
halloween spirit
```

gpt2.generate(sess, run_name='run1', nsamples=32, batch_size=16, destination_path="test", prefix="new year")

```
20020403
20020404
20040301
20040831
20041014
20041015
20050830
20051219
20051220
20060515
20071012
20081221
20081222
20101215
20101317
20110020
20110312
20110330
20120101
20120226
20120417
20120931
20130229
20130806
20131123
20140603
20140809
201408090
201408091
20150250
20150511
20150815
20150909
20160701
20160721
20170225
20170226
20180208
====================
new year british tourists face delays
new year for australias education
new year in crisis
new year in cycling
new year in music
new year in pictures
new year in politics
new year in queensland
new year in recognition
new year in review
new year in stock market
new year in the spotlight
new year of dairy
new year of fear
new year of the cattle
new year of the devil
new year of the dog
new year of the hmas
new year of the insects
new year of the jungle
new year of the river
new year of the seahorses
new year of the skink
new year of the tiger
new year of the turks
new year of the warriors
new year on
new year's baits
```

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='run1', nsamples=32, batch_size=16, destination_path="test", prefix="new year")

In [59]:
! cut -f1 -d',' test | uniq

new year in review
2011805
2011806
new year of the cattle
20170225
20170226
new year in cycling
20040301
200
new year in politics
20131123
new year in pictures
20081221
20081222
new year of the turks
20120417
201204
new year of dairy
20110020
new year of the warriors
20110312
new year of the river
20150909
201509
new year of the insects
20101215
new year in recognition
20140603
new year in queensland
20150511
201
new year of the skink
20120101
new year of the tiger
20051219
20051220
new year in review
20101317
new year on
20150250
new year of fear
20041014
20041015
20041
new year british tourists face delays
20130806
new year in music
20140809
201408090
201408091
new year of the devil
20160721
new year in politics
20130229
new year of the jungle
20060515
200
new year in crisis
20110330
201
new year for australias education
20071012
new year of the dog
20120931
201
new year in music
20180208
new year's baits
20120226
new year of the seahorses
20020403
20020404
new year of the hmas
20160

In [56]:
! cat test

halloween display
20160528,the drum thursday may 28
20160528,toddler encounter leaves boy in critical condition
20160528,tourist michael mcdonald parramatta
20160528,transport minister says a qantas flight grounded
20160528,tridents rammed into powerlines
20160528,uaco reaxing of royal hobart
20160528,un warns on the dangers of nuclear weapons as
20160528,unions back pay rise criticism
20160528,unions concerned about pay rise criticism
20160528,us approves nz's controversial anti
20160528,us considers military assistance to nz
20160528,vcat gets new call up
20160528,victorian government announces new hoon law
20160528,victoria water woes to be fixed with new
20160528,virgin lnp posts record profit
20160528,wa royal commission urges action on coal seam gas
20160528,wa premier murray to discuss royal commission report
20160528,wall st hits record high
20160528,wall street struggles to recover after sun
20160528,wet weather hits farmers with high prices
20160528,wach bromance
20160528,wac

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD WILLOUGHBY:
That, by the way, Clarence and I have done good side by side;
And yet side we, and he side we have done ill.

KING RICHARD II:
Why then 'tis done ill. O, how should I ease it?
Side with him and my brother, my sovereign!
Side wither away, and as night falls,
Like to the farthest morning to my last,
Side wither away, and as morning comes,
Like to the furthest afternoon to my last!
Side wither away, and as our fortunes turn,
Like to the furthest afternoon to our last!

QUEEN MARGARET:
What is this? counsel? counsel!

KING RICHARD II:
My queen and my heir, for half a mile and a half
She will glide this way, to be or no.

QUEEN MARGARET:
So stands the orchard here, for half a mile and a!

KING RICHARD II:
So stands the orchard here, to fence it, to!
Fashion it in her, like the hedgehog's net
LORD STANLEY:
What if I told you, in the hope of succor,
That I had lain a little while in your arms?

DUKE OF YORK:
No doubt, my lord.

QUEEN ELIZABETH:
'Tis a pity I should be coil'd 

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.